In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prep dataset

In [3]:
# prepare data
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [4]:
eval_dataset

Dataset({
    features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 406
})

In [5]:
import pandas as pd
eval_dataset = pd.DataFrame(eval_dataset)
eval_dataset.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


In [6]:
# prompt_template = "You are an experienced medical professional. Answer the following medial question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: {}"

# eval_dataset['prompt'] = eval_dataset.apply(lambda x: prompt_template.format(x['Open-ended Verifiable Question']), axis=1)
# eval_dataset.prompt[0]

In [7]:
# from datasets import Dataset
# ds = Dataset.from_pandas(eval_dataset[['prompt']])
# ds

## Set up model definition

In [8]:
# with open('/content/drive/My Drive/w266/project/266 Final Project/llama3pt2-1b-peft/adapter_config.json', 'r') as f:
#   l = f.readlines()

In [9]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the LoRA model
model_id = "erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-limited"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quantization_config,
#     token=userdata.get('HF_TOKEN'))

In [10]:
# Load tokenizer
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

1

In [11]:
tokenizer.get_chat_template()

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

## Eval of LoRA SFT (R = 32) model

In [12]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('notebook2'))


adapter_config.json:   0%|          | 0.00/868 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Device set to use cuda:0


In [13]:
fn_out = '/content/drive/MyDrive/w266_project/lora-sft-r32-limited-output-reformatted.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

In [14]:
for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):
  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content": "Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample}
  ]
  # prompt = tokenizer.apply_chat_template(prompt, tokenize=False)

  # prompt = "You are a medical expert. Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample
  for r in range(5):
    outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
    response = outputs[0]["generated_text"]
    response = response.replace('\n','\t')
    print(response)
    with open(fn_out, 'a') as f:
        f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 100:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in the 88-year-old woman is non-steroidal anti-inflammatory drugs (NSAIDs), such as naproxen. These medications can inhibit platelet aggregation, leading to bleeding, especially in the stomach.		Correctness rating: 1) Correct


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in this 88-year-old woman is due to the use of naproxen. Naproxen can lead to the formation of a gastric ulcer, which could bleed, especially in the elderly due to factors such as age-related physiological changes.		Correctness rating: 1) Correct


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The most likely diagnosis for the condition described in the question is Balanitis Plastica. This condition is characterized by recurrent scaly lesions on the glans penis, and it often follows a chronic course with slight hyperpigmentation, aligning well with the patient's symptoms.		**Correctness Rating:**	1. Correct


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely diagnosis for this condition, given the recurrent scaly lesions on the glans penis with a consistent location and healing with slight hyperpigmentation, is molluscum contagiosum. This viral infection is known to present with small, benign growths called mollusca that may appear at the same location and resolve with minimal scarring. 		The correctness of this answer is: **1. Correct**


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely diagnosis for the described condition is fixed condyloma acuminatum. This is due to the characteristic features of recurrent scaly lesions on the glans penis, which always appear at the same location and heal with slight hyperpigmentation, aligning with the pattern of condyloma acuminatum. In fact, fixed condyloma acuminatum is often the most appropriate diagnosis in such cases due to its propensity for persistent lesions.		Rating: 1. Correct
The most likely diagnosis for this condition is fixed drug eruption (FDE), characterized by the recurring nature of the lesions at a specific site and the presence of hyperpigmentation after each episode. While balanitis xerotica obstructive, which presents with similar symptoms, is a consideration, the specificity of the lesion's recurrence and site suggests that FDE is the more plausible diagnosis.		Rating: 1) Correct


In [ ]:
# prepare data for bootstrapping
fn_out = '/content/drive/MyDrive/w266_project/lora-sft-r32-output-reformatted-bootstrapping.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):
  if idx <= 101:
    continue
  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content":sample}
  ]
  # prompt = tokenizer.apply_chat_template(prompt, tokenize=False)

  # prompt = "You are a medical expert. Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample
  for r in range(5):
    outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
    response = outputs[0]["generated_text"]
    response = response.replace('\n','\t')
    print(response)
    with open(fn_out, 'a') as f:
        f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 201:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The symptoms described, including facial contortions, repetitive blinking, grunting, and bullying, suggest a possible diagnosis of Turette's Syndrome. In this context, the most effective medication for addressing these symptoms would typically involve a neuroleptic, specifically a typical antipsychotic. Risperdal (Risperidone) is a commonly used medication that targets these symptoms and is often recommended for treating Turette's Syndrome. Therefore, in this scenario, Risperdal is the medication most likely to be beneficial for this 11-year-old boy.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


For the 11-year-old boy exhibiting behavioral symptoms such as swearing, facial contortions, repetitive blinking, and grunting due to potential Tourette's syndrome, the most beneficial medication for managing his symptoms would be Risperdal (risperidone). This antipsychotic drug has been specifically approved by the FDA for the treatment of Tourette's syndrome. It works effectively in reducing tic severity and frequency, thus helping to alleviate the bullying issues he faces. While other medications like anti-seizure drugs and certain antidepressants might be considered, risperidone is the clear choice for addressing the specific symptoms associated with this condition.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Based on the clinical features you described, the most likely anatomic site implicated in this woman's condition is the neuromuscular junction. Her fatigue, early muscle weakness, and decrease in handgrip strength with repetitive testing are suggestive of a problem at this junction, where the nerve signal interacts with the muscle. This scenario is reminiscent of myasthenia gravis, where an autoimmune response often affects the neuromuscular junction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The primary mechanism of action of the hormone atrial natriuretic peptide (ANP) on the kidneys is the promotion of natriuresis, or the excretion of sodium. ANP does this by enhancing the activity of several renal hormones: (1) aldosterone antagonism, which prevents the body from retaining sodium; (2) inhibiting the renin-angiotensin-aldosterone system (RAAS), which suppresses aldosterone and reduces sodium retention; and (3) increasing the production of natriuretic hormones like brain natriuretic peptide (BNP), which further supports sodium excretion. Overall, ANP helps the kidneys to eliminate excess sodium and thereby reduces blood volume, leading to a decrease in blood pressure. This hormone plays a vital role in maintaining fluid and electrolyte balance.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The hormone that is positively correlated with left atrial pressure and is likely released by atrial myocytes in the context of acute decompensated congestive heart failure is atrial natriuretic peptide (ANP). The primary mechanism of action of ANP on the kidneys is the inhibition of the actions of the hormone aldosterone. Typically, aldosterone promotes sodium retention by the kidneys, which helps conserve salt and water, especially in hypertensive states. However, during heart failure, the increased atrial pressure leads to increased ANP release. ANP prevents the production of aldosterone by blocking the actions of angiotensin II on the adrenal glands. This results in reduced sodium reabsorption by the kidneys and increased excretion of sodium in the urine, thereby counteracting the sodium-retaining effects of aldosterone and supporting cardiovascular stability.
